In [1]:
import os
#os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\amazon-fine-food-reviews')
os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\AmazonFineFood\\database.sqlite')
import pandas as pd
import numpy as np
import nltk
import spacy
import sqlite3
import spacy


In [2]:
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10""", con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [22]:
for i in range(len(filtered_data['Text'])):
    doc=nlp(filtered_data['Text'][i])
    filter_list=[]
    for token in doc:
        if token.pos_!='NOUN':
            filter_list.append(token)   
    print(filter_list)
    filtered_data['Text'][i]=' '.join(filter_list) 
       
        

[I, have, bought, several, of, the, Vitality, canned, and, have, found, them, all, to, be, of, good, ., The, looks, more, like, a, than, a, processed, and, it, smells, better, ., My, Labrador, is, finicky, and, she, appreciates, this, better, than,  , most, .]


TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [20]:
a=[I, have, bought, several, of, the, Vitality, canned, and, have, found, them, all, to, be, of, good, ., The, looks, more, like, a, than, a, processed, and, it, smells, better, My, Labrador, is, finicky, and, she, appreciates, this, better, than,  , most]

SyntaxError: invalid syntax (<ipython-input-20-2f0f2aa8257f>, line 1)